# Added mass - two bodies

In [1]:
using ViscousFlow

In [2]:
using Plots
pyplot()
clibrary(:colorbrewer)
default(grid = false)

In [3]:
using LinearAlgebra

### First, some generalities

In [4]:
𝐞₁ = [1;0];
𝐞₂ = [0;1];

#### First create a grid and a Laplacian operator on it

In [21]:
Lx = 3.5;
Ly = 2.0;
nx = 512;
dx = Lx/(nx-2);
ny = Int(ceil(Ly/dx))+2;
w = Nodes(Dual,(nx,ny));

In [6]:
L = plan_laplacian(size(w),with_inverse=true)

Discrete Laplacian (and inverse) on a (nx = 512, ny = 294) grid with spacing 1.0

#### Now create the circular shapes, with discrete points and associated regularization and interpolation operator

In [7]:
n = 196

R = 0.5
body_1 = Ellipse(R,R,n)
body_2 = Ellipse(R,R,n)

# Find the minimum arc length
ds = minimum(Bodies.dlength(body_1))

# Move the bodies
xc_1 = 1.0; yc_1 = 1.0
xc_2 = 2.5; yc_2 = 1.0
T_1 = RigidTransform((xc_1,yc_1),0.0)
T_1(body_1)
T_2 = RigidTransform((xc_2,yc_2),0.0)
T_2(body_2)

# Area of the circles
𝒱 = π*R^2

0.7853981633974483

#### Set up the coordinates and other useful vectors

In [8]:
X = VectorData([body_1.x;body_2.x],[body_1.y;body_2.y])
f = ScalarData(X);

In [9]:
println("Ratio of arc spacing to cell size = ",ds/dx)

Ratio of arc spacing to cell size = 2.3354863689681555


In [10]:
E = Regularize(X,dx;issymmetric=true)
Hmat,Emat = RegularizationMatrix(E,f,w);

#### And now create the saddle-point system

In [11]:
L⁻¹(w::T) where {T} = L\w
PS = SaddleSystem((w,f),(L⁻¹,Hmat,Emat),issymmetric=true,isposdef=true)

Saddle system with 392 constraints and
   State of type Nodes{Dual,512,294}
   Force of type ScalarData{392}


#### Create some data structures for general use

In [12]:
ψb = ScalarData(X)
w = Nodes(Dual,(nx,ny))
ψ = Nodes(Dual,w);

### Solve flow generated by a translating cylinder

In [13]:
U_1 = 1;
V_1 = 0;
U_2 = 0;
V_2 = 0;

In [14]:
ψb[1:n] .= U_1*(body_1.y .- body_1.cent[2]) - V_1*(body_1.x .- body_1.cent[1])
ψb[n+1:end] .= U_2*(body_2.y .- body_2.cent[2]) - V_2*(body_2.x .- body_2.cent[1])
@time ψ,f = PS\(w,ψb)

  0.075479 seconds (225 allocations: 19.617 MiB, 12.12% gc time)


(Dual nodes in a (nx = 512, ny = 294) cell grid
  Number of Dual nodes: (nx = 512, ny = 294), [-5.09814e-8, 0.00112404, 0.00224515, 0.00336537, 0.00448101, 0.0055813, 0.00667879, 0.0077678, 0.00884042, 0.00988996  …  0.000374258, 0.000336943, 0.000299339, 0.000261806, 0.000224691, 0.000187494, 0.000150162, 0.000113127, 7.60645e-5, 3.89393e-5])

#### Plot the result

In [15]:
xg,yg = coordinates(ψ,dx=dx)
plot(xg,yg,ψ,linecolor=:black,levels=range(-1,1,length=200))
plot!(body_1,fillcolor=:grey,fillrange=0,fillalpha=1.0,linecolor=:black)
plot!(body_2,fillcolor=:grey,fillrange=0,fillalpha=1.0,linecolor=:black)

#### Check added mass coefficient

In [16]:
"""
integrate(f::Function,xmin::Real,xmax::Real,N::Int,name::String) -> Real
Integrate the function `f` between the limits `xmin` to `xmax` with `N` intervals using either the `"trapezoidal"` or `"simpson"` rule specified in `name`. The solution is returned as a Real value.
"""
function surface_integrate(body::Body{N},integrand::Array{Float64,1}) where {N}
    func = Array{Float64,1}(undef, N+1)
    func[1:end-1] .= integrand
    func[end] = integrand[1]
    s = sum(Bodies.dlength(body).*(func[1:end-1] + func[2:end]))
    return s
end

surface_integrate

$\boldsymbol{x}\times\boldsymbol{\gamma}_\mathrm{bt}^{(j)}$

In [17]:
integrand_1_x = (f[1:n]./Bodies.dlength(body_1)).*(body_1.y);
integrand_1_y = -(f[1:n]./Bodies.dlength(body_1)).*(body_1.x);
integrand_2_x = (f[n+1:end]./Bodies.dlength(body_2)).*(body_2.y);
integrand_2_y = -(f[n+1:end]./Bodies.dlength(body_2)).*(body_2.x);

In [18]:
p1 = plot(integrand_1_x)
p2 = plot(integrand_1_y)
p3 = plot(p1,p2,layout=(1,2),size=[800,400])

$P_{\mathrm{bt}}^{(j)}=-\mathcal{V}_{\mathrm{b}} \tilde{\boldsymbol{e}}_{j}+\frac{1}{n_{\mathrm{d}}-1} \int_{S_{\mathrm{b}}} \boldsymbol{x}\times\boldsymbol{\gamma}_\mathrm{bt}^{(j)}\mathrm{d} S$

In [19]:
P¹ = -𝒱*𝐞₁ + 0.5*[surface_integrate(body_1,integrand_1_x);surface_integrate(body_1,integrand_1_y)]

2-element Array{Float64,1}:
 0.8245219735222884   
 0.0002370448730883972

In [20]:
M₁₁ = dot(P¹,𝐞₁)

0.8245219735222884

Compare the generated vortex sheet `f/ds` with the exact solution

In [110]:
θ = range(0,2π,length=n+1)
fex = 2*sin.(θ[1:n]);
errinf = LinearAlgebra.norm(f[1:n]./ds-fex,Inf)

0.10410960089278642

In [111]:
plot(f[1:n]./ds,label="Numerical")
plot!(fex,label="Exact")